In [2]:
#referencia https://medium.com/towards-artificial-intelligence/multi-class-text-classification-using-pyspark-mllib-doc2vec-dbfcee5b39f2
import findspark
findspark.init() 
from pyspark.sql import SparkSession
import os


MAX_MEMORY = "8g"
spark = SparkSession.builder \
                    .master("local") \
                    .appName('va_repositorio')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.jars", "E:\\va\\driverdb\\postgresql-42.2.5.jre6") \
                    .getOrCreate()

In [4]:
tabla="proyecto.va_repositorio_inicial"
df  = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/va") \
    .option("dbtable", tabla) \
    .option("user", "postgres") \
    .option("password", "david") \
    .option("driver", "org.postgresql.Driver") \
    .load()
df.printSchema()

root
 |-- periodo: string (nullable = true)
 |-- estu_mcpio_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_estadocivil: string (nullable = true)
 |-- estu_etnia: string (nullable = true)
 |-- estu_limita_motriz: string (nullable = true)
 |-- estu_limita_invidente: string (nullable = true)
 |-- estu_limita_sordoconinterprete: string (nullable = true)
 |-- estu_limita_sordosininterprete: string (nullable = true)
 |-- estu_limita_sordoceguera: string (nullable = true)
 |-- estu_porcentajecreditosaprob: string (nullable = true)
 |-- estu_valormatriculauniversidad: string (nullable = true)
 |-- estu_pagomatriculabeca: string (nullable = true)
 |-- estu_pagomatriculacredito: string (nullable = true)
 |-- estu_pagomatriculapadres: string (nullable = true)
 |-- estu_pagomatriculapropio: string (nullable = true)
 |-- estu_tomo_cursopreparacion: string (nullable = true)
 |-- estu_cursodocentesies: string (nullable = true)
 |-- estu_cursoiesapoyoexterno: str

In [8]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lower, col
import re
for c in df.columns: 
    df=(df
         .withColumn(c, lower(col(c)))
         .withColumn(c, regexp_replace(c, ' ', '_'))
         .withColumn(c, regexp_replace(c, ',', ''))
         .withColumn(c, regexp_replace(c, 'á', 'a'))
         .withColumn(c, regexp_replace(c, 'é', 'e'))
         .withColumn(c, regexp_replace(c, 'é', 'e'))
         .withColumn(c, regexp_replace(c, 'í', 'i'))
         .withColumn(c, regexp_replace(c, 'ó', 'o'))
         .withColumn(c, regexp_replace(c, 'ú', 'u'))
         .withColumn(c, regexp_replace(c, 'ñ', 'n')))

In [32]:
df=(df
         .withColumn("estu_mcpio_reside", regexp_replace("estu_mcpio_reside", 'ü', 'u'))
         .withColumn("estu_estadocivil", regexp_replace("estu_estadocivil", '/', '_'))
         .withColumn("estu_trabaja_actualmente", regexp_replace("estu_trabaja_actualmente", '/', '_'))
         .withColumn("estu_porcentajecreditosaprob", regexp_replace("estu_porcentajecreditosaprob", '%', '_por_centaje')))

In [38]:
import re

string = "Hey! What's up bro?"
new_string = re.sub(r"[^a-zA-Z0-9]","",string)
print(new_string)

TypeError: expected string or bytes-like object

In [37]:
df.groupby("fami_educacionpadre").count().show()



+--------------------+-----+
| fami_educacionpadre|count|
+--------------------+-----+
|educacion_profesi...|  920|
|           no_aplica|    9|
|secundaria_(bachi...|  825|
|educacion_profesi...|  188|
|   primaria_completa| 1228|
|           postgrado|  376|
|tecnica_o_tecnolo...|  164|
|secundaria_(bachi...| 2029|
| primaria_incompleta| 1798|
|             no_sabe|   95|
|             ninguno|  292|
|tecnica_o_tecnolo...|  487|
|                    |  188|
+--------------------+-----+



In [12]:
df.write.jdbc(url="jdbc:postgresql://localhost:5432/va"
                  "?user=postgres&password=david",
              table="proyecto.va_repositorio_v2",
              mode="overwrite",
              properties={"driver": 'org.postgresql.Driver'})